In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.utils import image_dataset_from_directory

In [55]:
train_ds = image_dataset_from_directory(
    "../data/human-and-non-human/training_set/training_set",
    validation_split=0.3,
    subset="training",
    seed=123,
    image_size=(250,250),
    batch_size=32,
    labels="inferred",
)
train_ds

Found 8017 files belonging to 2 classes.
Using 5612 files for training.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 250, 250, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [54]:
val_ds = image_dataset_from_directory(
 "../data/human-and-non-human/training_set/training_set",
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(250,250),
  batch_size=32,
  labels="inferred",
)
val_ds

Found 8017 files belonging to 2 classes.
Using 2405 files for validation.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 250, 250, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [9]:
from keras import Sequential
from keras import layers
from keras.layers import Rescaling

In [56]:

from keras import layers, models

# Load pretrained MobileNetV2
base_model = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(250, 250, 3))

# Freeze the base model (not trainable)
base_model.trainable = False

# Add custom layers on top for your specific task
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # For binary classification: human (1) vs not human (0)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Now, you can fine-tune it or train on your human vs not-human dataset
model.fit(train_ds, epochs=5, validation_data=val_ds)


C:\Users\HP\AppData\Local\Temp\ipykernel_13808\540324685.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(250, 250, 3))


Epoch 1/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 134s 741ms/step - accuracy: 0.8858 - loss: 0.2568 - val_accuracy: 0.9609 - val_loss: 0.1045
Epoch 2/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 137s 782ms/step - accuracy: 0.9701 - loss: 0.0805 - val_accuracy: 0.9696 - val_loss: 0.0745
Epoch 3/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 146s 830ms/step - accuracy: 0.9844 - loss: 0.0482 - val_accuracy: 0.9613 - val_loss: 0.0927
Epoch 4/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 119s 676ms/step - accuracy: 0.9880 - loss: 0.0369 - val_accuracy: 0.9742 - val_loss: 0.0671
Epoch 5/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 78s 441ms/step - accuracy: 0.9916 - loss: 0.0291 - val_accuracy: 0.9825 - val_loss: 0.0495


In [59]:

test_training_ds = image_dataset_from_directory(
  "../data/human-and-non-human/test_set/test_set",
  seed=123,
  image_size=(250,250),
  batch_size=32,
  labels="inferred",

)
test_training_ds

Found 2723 files belonging to 2 classes.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 250, 250, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [60]:
model.evaluate(test_training_ds)

86/86 ━━━━━━━━━━━━━━━━━━━━ 27s 308ms/step - accuracy: 0.9737 - loss: 0.0705


[0.06365882605314255, 0.977230966091156]

In [61]:
train_ds.class_names

['humans', 'non-humans']

In [71]:
import cv2 as cv
import numpy as np

# Load grayscale image
image = cv.imread("../data/human-and-non-human/sarbesh.jpg")

# Resize and normalize
resized_image = cv.resize(image, (250, 250))
resized_image = np.array(resized_image)

resized_image = np.expand_dims(resized_image , axis=0)

# Now this will work
prediction = model.predict(resized_image)
if prediction>0.5:
    print("human")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
human


In [68]:
model.predict(resized_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


array([[0.97927094]], dtype=float32)

In [73]:
import joblib
joblib.dump(model,"human_recog_model.joblib")

['human_recog_model.joblib']